In [1]:
# Install a conda package in the current Jupyter kernel
# https://jakevdp.github.io/blog/2017/12/05/installing-python-packages-from-jupyter/
import sys
!{sys.executable} -m pip install -r requirements.txt # This installs all the required packages

# Analysis of COVID19 Time Series Summary - Workbook 1

The dataset we will be using is the Time Series Summary collected by the Johns Hopkins University. The data is updated on the Github repository [here](https://github.com/CSSEGISandData/COVID-19).

We will be looking into three files:
  * [time_series_covid19_confirmed_global.csv](https://github.com/CSSEGISandData/COVID-19/blob/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv)
  * [time_series_covid19_deaths_global.csv](https://github.com/CSSEGISandData/COVID-19/blob/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv)
  * [time_series_covid19_recovered_global.csv](https://github.com/CSSEGISandData/COVID-19/blob/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv)

As stated in the Github repo, the "Three time series tables are for the global confirmed cases, recovered cases and deaths. Australia, Canada and China are reported at the province/state level. Dependencies of the Netherlands, the UK, France and Denmark are listed under the province/state level. The US and other countries are at the country level..."

Since we have daily numbers of the pandemic on a global, we can ask numerous questions once we have explored, preprocessed and analysed the data. Let us have explore one of the files to gather a preliminary idea of the available data.

The following methods from `Pandas` library is recommended to be used with this workbook:
* read_csv
* DataFrame.sample
* DataFrame.head
* DataFrame.tail
* DataFrame.info
* DataFrame.describe
* DataFrame.columns.values
* DataFrame.melt
* DataFrame.merge
* DataFrame.rename
* to_datetime
* DataFrame.Date
* Series.dt.date
* DataFrame.isna
* Series.fillna
* DataFrame.groupby
* DataFrame.sum
* DataFrame.diff
* Series.unique
* Series.str.contains
* Series.reset_index
* DataFrame.reset_index
* DataFrame.to_csv

In [2]:
%matplotlib inline
import pandas as pd
import numpy as np
import datetime as dt

# Used to ignore warnings arising from pandas module
import warnings
warnings.filterwarnings('ignore')

### Read the csv files

In [3]:
try:
    # Use url to download the files instead of using older csvs
    url_deaths = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv"
    url_confirmed = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv"
    url_recovered = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv" 

    df_confirmed = pd.read_csv(url_confirmed)
    df_deaths = pd.read_csv(url_deaths)
    df_recovered = pd.read_csv(url_recovered)

except requests.HTTPError as e:
    print(f"At least one of the urls do not work.")
    
    df_confirmed = pd.read_csv('time_series_covid19_confirmed_global.csv') 
    df_deaths = pd.read_csv('time_series_covid19_deaths_global.csv') 
    df_recovered = pd.read_csv('time_series_covid19_recovered_global.csv') 


### Initial exploration of the dataframe

In [4]:
display(df_confirmed.head())
display(df_confirmed.describe())
display(df_confirmed.info())
display(df_confirmed.sample(5))

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,6/6/20,6/7/20,6/8/20,6/9/20,6/10/20,6/11/20,6/12/20,6/13/20,6/14/20,6/15/20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,...,19551,20342,20917,21459,22142,22890,23546,24102,24766,25527
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,...,1232,1246,1263,1299,1341,1385,1416,1464,1521,1590
2,NaN,Algeria,28.0339,1.6596,0,0,0,0,0,0,...,10050,10154,10265,10382,10484,10589,10698,10810,10919,11031
3,NaN,Andorra,42.5063,1.5218,0,0,0,0,0,0,...,852,852,852,852,852,852,853,853,853,853
4,NaN,Angola,-11.2027,17.8739,0,0,0,0,0,0,...,88,91,92,96,113,118,130,138,140,142


,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,...,6/6/20,6/7/20,6/8/20,6/9/20,6/10/20,6/11/20,6/12/20,6/13/20,6/14/20,6/15/20
count,266.000000,266.000000,266.000000,266.000000,266.000000,266.000000,266.000000,266.000000,266.000000,266.000000,...,2.660000e+02,2.660000e+02,2.660000e+02,2.660000e+02,2.660000e+02,2.660000e+02,2.660000e+02,2.660000e+02,2.660000e+02,2.660000e+02
mean,21.148043,22.538640,2.086466,2.458647,3.537594,5.390977,7.962406,11.003759,20.969925,23.180451,...,2.594981e+04,2.637496e+04,2.676449e+04,2.722817e+04,2.773058e+04,2.825084e+04,2.873782e+04,2.924391e+04,2.974596e+04,3.020474e+04
std,24.909990,70.466256,27.279200,27.377862,34.083035,47.434934,66.289178,89.313757,219.187744,220.524977,...,1.333309e+05,1.349130e+05,1.363934e+05,1.382464e+05,1.402889e+05,1.424252e+05,1.445871e+05,1.467117e+05,1.484660e+05,1.502466e+05
min,-51.796300,-135.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,6.907750,-18.093125,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.572500e+02,1.572500e+02,1.572500e+02,1.580000e+02,1.625000e+02,1.650000e+02,1.710000e+02,1.722500e+02,1.760000e+02,1.760000e+02
50%,23.488100,20.972650,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,9.535000e+02,9.710000e+02,9.820000e+02,9.825000e+02,9.825000e+02,9.830000e+02,1.005000e+03,1.005000e+03,1.005000e+03,1.005000e+03
75%,41.143200,77.191525,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,7.094500e+03,7.117250e+03,7.427000e+03,7.706750e+03,8.005750e+03,8.194000e+03,8.334500e+03,8.452500e+03,8.586500e+03,8.608750e+03
max,71.706900,178.065000,444.000000,444.000000,549.000000,761.000000,1058.000000,1423.000000,3554.000000,3554.000000,...,1.926538e+06,1.944309e+06,1.961781e+06,1.979868e+06,2.000702e+06,2.023590e+06,2.048986e+06,2.074526e+06,2.094058e+06,2.114026e+06


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 266 entries, 0 to 265
Columns: 150 entries, Province/State to 6/15/20
dtypes: float64(2), int64(146), object(2)
memory usage: 311.8+ KB


None

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,6/6/20,6/7/20,6/8/20,6/9/20,6/10/20,6/11/20,6/12/20,6/13/20,6/14/20,6/15/20
124,NaN,Guinea,9.9456,-9.6966,0,0,0,0,0,0,...,4117,4117,4216,4258,4258,4372,4426,4484,4532,4572
164,NaN,Namibia,-22.9576,18.4904,0,0,0,0,0,0,...,29,29,31,31,31,31,31,32,32,32
245,Yukon,Canada,64.2823,-135.0000,0,0,0,0,0,0,...,11,11,11,11,11,11,11,11,11,11
226,NaN,Uzbekistan,41.3775,64.5853,0,0,0,0,0,0,...,4094,4331,4440,4520,4623,4741,4869,4966,5080,5263
91,NaN,Czechia,49.8175,15.4730,0,0,0,0,0,0,...,9567,9628,9697,9751,9824,9855,9938,9991,10024,10064


### List all the column values in a dataframe

Let's check what columns we have to deal with.

In [5]:
df_confirmed.columns.values

array(['Province/State', 'Country/Region', 'Lat', 'Long', '1/22/20',
       '1/23/20', '1/24/20', '1/25/20', '1/26/20', '1/27/20', '1/28/20',
       '1/29/20', '1/30/20', '1/31/20', '2/1/20', '2/2/20', '2/3/20',
       '2/4/20', '2/5/20', '2/6/20', '2/7/20', '2/8/20', '2/9/20',
       '2/10/20', '2/11/20', '2/12/20', '2/13/20', '2/14/20', '2/15/20',
       '2/16/20', '2/17/20', '2/18/20', '2/19/20', '2/20/20', '2/21/20',
       '2/22/20', '2/23/20', '2/24/20', '2/25/20', '2/26/20', '2/27/20',
       '2/28/20', '2/29/20', '3/1/20', '3/2/20', '3/3/20', '3/4/20',
       '3/5/20', '3/6/20', '3/7/20', '3/8/20', '3/9/20', '3/10/20',
       '3/11/20', '3/12/20', '3/13/20', '3/14/20', '3/15/20', '3/16/20',
       '3/17/20', '3/18/20', '3/19/20', '3/20/20', '3/21/20', '3/22/20',
       '3/23/20', '3/24/20', '3/25/20', '3/26/20', '3/27/20', '3/28/20',
       '3/29/20', '3/30/20', '3/31/20', '4/1/20', '4/2/20', '4/3/20',
       '4/4/20', '4/5/20', '4/6/20', '4/7/20', '4/8/20', '4/9/20',
       '4

We can see that $4^{th}$ column onwards are the dates starting from Jan 22, 2020 till the latest date. The Github repository should have data updated once a day. This structure should be repeated in the remainder of the csv files. Before doing analysis and comparision between the *confirmed*, *recovered* and *death* cases, we need to merge, clean and aggregate the data as we seem fit. We rarely get data in a clean format, or a format we can readily process, so the aforementioned techniques are very useful when dealing with raw data. Ultimately, we wish to have a single csv with the *confirmed*, *recovered* and *death* cases, a dataframe that looks somewhat like this.

![Random rows of semi-final csv file](./img/semi_final.png)

In order to process dates in rows, we can use the `melt` method of a dataframe.

In [6]:
dates = df_confirmed.columns[4:].values

def df_wide_to_long(dataframe, value_name):
    return dataframe.melt(
        id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], 
        var_name='Date', 
        value_name=value_name,
        value_vars=dates, 
    )

# Using an appropriate value for each column, 
# convert the three dataframes from wide to long format using 
# the df_wide_to_long function

# Your code here
df_confirmed_longf = df_wide_to_long(df_confirmed, "confirmed")
df_deaths_longf = df_wide_to_long(df_deaths, "deaths")
df_recovered_longf = df_wide_to_long(df_recovered, "recovered")

In [7]:
df_confirmed_longf.sample(5)

,Province/State,Country/Region,Lat,Long,Date,confirmed
28535,Shandong,China,36.3427,118.1498,5/8/20,788
7736,NaN,Belarus,53.7098,27.9534,2/20/20,0
21580,NaN,Cameroon,3.8480,11.5021,4/12/20,820
15275,Saint Barthelemy,France,17.9000,-62.8333,3/19/20,3
20148,NaN,Slovenia,46.1512,14.9955,4/6/20,1021


## Merge the dataframes

Now we need to merge the dataframes from the confirmed, recovered and death csv files into a single dataframe. The dataframe should look as following:

![Combined dataframe](./img/combined_df.png)

In [8]:
# Merging df_combined and deaths_df_long
df_combined = df_confirmed_longf.merge(
  right=df_deaths_longf, 
  how='left',
  on=['Province/State', 'Country/Region', 'Date', 'Lat', 'Long']
)

# Merging df_combined and recovered_df_long
df_combined = df_combined.merge(
  right=df_recovered_longf, 
  how='left',
  on=['Province/State', 'Country/Region', 'Date', 'Lat', 'Long']
)

In [9]:
df_combined.sample(5)

,Province/State,Country/Region,Lat,Long,Date,confirmed,deaths,recovered
14713,NaN,Congo (Brazzaville),-4.0383,21.7587,3/17/20,1,0,0.0
31518,NaN,Iceland,64.9631,-19.0208,5/19/20,1802,10,1789.0
15381,Channel Islands,United Kingdom,49.3723,-2.3644,3/19/20,11,0,0.0
11198,NaN,Bolivia,-16.2902,-63.5887,3/4/20,0,0,0.0
22689,Xinjiang,China,41.1129,85.2401,4/16/20,76,3,73.0


In [10]:
# Also, it is somewhat inconvenient to deal with '/' in column header and to deal with upper cases, so lets rename these headers
renamed_columns = {
    "Province/State": "state", 
    "Country/Region": "country",
}

df_combined = df_combined.rename(columns=renamed_columns)

# Convert all column headers to lower case
df_combined.columns = map(str.lower, df_combined.columns)

In [11]:
df_combined.columns.values

array(['state', 'country', 'lat', 'long', 'date', 'confirmed', 'deaths',
       'recovered'], dtype=object)

### Convert Date from string to datetime

Before we can use date values, we need to use the `to_datetime` method to convert the **Date** values to date values (e.g. `2020-04-22`). This makes it easier to analyse and plot the data relating to the time series.

In [12]:
df_combined["date"] = pd.to_datetime(df_combined["date"]).dt.date

In [13]:
df_combined.sample(5)

,state,country,lat,long,date,confirmed,deaths,recovered
16745,NaN,Burundi,-3.3731,29.9189,2020-03-24,0,0,0.0
8824,NaN,Central African Republic,6.6111,20.9394,2020-02-24,0,0,0.0
34261,NaN,Turkey,38.9637,35.2433,2020-05-29,162120,4489,125963.0
37249,New South Wales,Australia,-33.8688,151.2093,2020-06-10,3117,48,2738.0
34659,Xinjiang,China,41.1129,85.2401,2020-05-31,76,3,73.0


### Dealing with `na` values

Let's see the number of missing values in the combined dataframe. 

In [14]:
df_combined.isna().sum()

state        27010
country          0
lat              0
long             0
date             0
confirmed        0
deaths           0
recovered     2482
dtype: int64

It is understandable to see missing values in state, since some countries may not record cases by states, while some countries do not actually have states altogether.

It seems like the "recovered" columns seems to have a fair amount of missing values. Some of the reasons for this could be: 
- no patients have recovered
- the countries are not recording this information

Replace the `na` values with appropriate values. What would be a replacement for `na` values in the *Recovered* column appropriate for numerical operations?

In [15]:
df_combined["recovered"] = df_combined["recovered"].fillna(0)
df_combined.isna().sum() # check to see all the missing values have been replaced

state        27010
country          0
lat              0
long             0
date             0
confirmed        0
deaths           0
recovered        0
dtype: int64

## Cleaning the Country and State values

Run the following function calls to find the set of State and Country values.

In [16]:
df_combined.state.unique()

array([nan, 'Australian Capital Territory', 'New South Wales',
       'Northern Territory', 'Queensland', 'South Australia', 'Tasmania',
       'Victoria', 'Western Australia', 'Alberta', 'British Columbia',
       'Grand Princess', 'Manitoba', 'New Brunswick',
       'Newfoundland and Labrador', 'Nova Scotia', 'Ontario',
       'Prince Edward Island', 'Quebec', 'Saskatchewan', 'Anhui',
       'Beijing', 'Chongqing', 'Fujian', 'Gansu', 'Guangdong', 'Guangxi',
       'Guizhou', 'Hainan', 'Hebei', 'Heilongjiang', 'Henan', 'Hong Kong',
       'Hubei', 'Hunan', 'Inner Mongolia', 'Jiangsu', 'Jiangxi', 'Jilin',
       'Liaoning', 'Macau', 'Ningxia', 'Qinghai', 'Shaanxi', 'Shandong',
       'Shanghai', 'Shanxi', 'Sichuan', 'Tianjin', 'Tibet', 'Xinjiang',
       'Yunnan', 'Zhejiang', 'Faroe Islands', 'Greenland',
       'French Guiana', 'French Polynesia', 'Guadeloupe', 'Mayotte',
       'New Caledonia', 'Reunion', 'Saint Barthelemy', 'St Martin',
       'Martinique', 'Aruba', 'Curacao', 'Sint

In [17]:
df_combined.country.unique()

array(['Afghanistan', 'Albania', 'Algeria', 'Andorra', 'Angola',
       'Antigua and Barbuda', 'Argentina', 'Armenia', 'Australia',
       'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain', 'Bangladesh',
       'Barbados', 'Belarus', 'Belgium', 'Benin', 'Bhutan', 'Bolivia',
       'Bosnia and Herzegovina', 'Brazil', 'Brunei', 'Bulgaria',
       'Burkina Faso', 'Cabo Verde', 'Cambodia', 'Cameroon', 'Canada',
       'Central African Republic', 'Chad', 'Chile', 'China', 'Colombia',
       'Congo (Brazzaville)', 'Congo (Kinshasa)', 'Costa Rica',
       "Cote d'Ivoire", 'Croatia', 'Diamond Princess', 'Cuba', 'Cyprus',
       'Czechia', 'Denmark', 'Djibouti', 'Dominican Republic', 'Ecuador',
       'Egypt', 'El Salvador', 'Equatorial Guinea', 'Eritrea', 'Estonia',
       'Eswatini', 'Ethiopia', 'Fiji', 'Finland', 'France', 'Gabon',
       'Gambia', 'Georgia', 'Germany', 'Ghana', 'Greece', 'Guatemala',
       'Guinea', 'Guyana', 'Haiti', 'Holy See', 'Honduras', 'Hungary',
       'Iceland', 'India

We discover that the there are coronavirus cases reported from three cruise ships: *Grand Princess*, *Diamond Princess* and *MS Zaandam*. These data need to be extracted and treated differently as they do not necessarily relate to State and Region over time.

We can do the this in the following steps:
  * get the rows of the dataframe containing the cruise ship date
  * combine the rows into their own dataframe
 
If we wish to combine the states and visualise the data from a Country or State perspective, it may also be a good idea remove the ship data from the *combined_df*.

In [19]:
ship_rows = df_combined['state'].str.contains('Grand Princess') | df_combined['state'].str.contains('Diamond Princess') | \
                df_combined['country'].str.contains('Diamond Princess') | df_combined['country'].str.contains('MS Zaandam')
ship_df = df_combined[ship_rows]

ship_df.sample(5)

,state,country,lat,long,date,confirmed,deaths,recovered
19137,NaN,MS Zaandam,0.0000,0.0000,2020-04-02,9,2,0.0
11260,NaN,Diamond Princess,0.0000,0.0000,2020-03-04,706,6,10.0
22913,Grand Princess,Canada,37.6489,-122.6655,2020-04-17,13,0,0.0
37277,Grand Princess,Canada,37.6489,-122.6655,2020-06-10,13,0,0.0
7433,NaN,MS Zaandam,0.0000,0.0000,2020-02-18,0,0,0.0


Remove the rows containing the ship data

In [20]:
df_combined = df_combined[~(ship_rows)]

## Number of active cases

We can retrieve the number of active cases using the following equation:
$$#active_cases = #confirmed_cases - (#deaths + #recovered)$$

Let's use this equation to create a new column, **Active**, that tracks the daily number of active cases per location.

In [21]:
df_combined["active"] = df_combined.confirmed - (df_combined.deaths + df_combined.recovered)

In [23]:
# Sample the dataframe to probe if the column has been created correctly
df_combined.sample(5)

,state,country,lat,long,date,confirmed,deaths,recovered,active
23998,Hebei,China,39.5490,116.1306,2020-04-21,328,6,317.0,5.0
6311,NaN,Senegal,14.4974,-14.4524,2020-02-14,0,0,0.0,0.0
11095,NaN,Saint Lucia,13.9094,-60.9789,2020-03-03,0,0,0.0,0.0
29369,Guadeloupe,France,16.2500,-61.5833,2020-05-11,154,13,104.0,37.0
12620,NaN,Gambia,13.4432,-15.3101,2020-03-09,0,0,0.0,0.0


## Data aggregation

The data can be aggregated in a multitude of ways. In this case, we will be aggregate data Country wise and group them by Date and Country. The dataframe should look as follows:

![Aggregation by country](./img/aggregation_country.png)

In [24]:
df_country_grouped = df_combined.groupby(['date', 'country'])['lat', 'long','confirmed', 'deaths', 'recovered', 'active'].sum().reset_index()

In [25]:
df_country_grouped.sample(10)

,date,country,lat,long,confirmed,deaths,recovered,active
12934,2020-03-31,Liechtenstein,47.140000,9.550000,68,0,0.0,68.0
22329,2020-05-21,Austria,47.516200,14.550100,16404,633,14951.0,820.0
11071,2020-03-21,Lesotho,-29.609988,28.233608,0,0,0.0,0.0
24357,2020-05-31,Uruguay,-32.522800,-55.765800,823,22,685.0,116.0
10575,2020-03-18,Suriname,3.919300,-56.027800,1,0,0.0,1.0
12010,2020-03-26,Maldives,3.202800,73.220700,13,0,8.0,5.0
6410,2020-02-25,Japan,36.000000,138.000000,170,1,22.0,147.0
3391,2020-02-09,Croatia,45.100000,15.200000,0,0,0.0,0.0
19880,2020-05-07,Tajikistan,38.861034,71.276093,461,12,0.0,449.0
19320,2020-05-04,Syria,34.802075,38.996815,44,3,0.0,41.0


Lets write the dataframes we created throughout the notebook into respective csv file according to aggregation for future usage.
- COVID19_time_series_ship
- COVID19_time_series_country_state
- COVID19_time_series_country

In [27]:
ship_df.to_csv('COVID19_time_series_ship.csv', index=False)
df_combined.to_csv('COVID19_time_series_country_state.csv', index=False)
df_country_grouped.to_csv('COVID19_time_series_country.csv', index=False)